# Q15题 感知机算法实现

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [101]:
# 导入并处理数据
def loaddata(filename):
    data = np.loadtxt(filename)
    row,col = np.shape(data)
    X = np.concatenate([np.ones((row,1)), data[:,:col-1]], axis =1)
    y = data[:, -1].reshape((row, 1))
    return X,y
    

In [102]:
filename = 'hw1_15_train.dat'
X,y = loaddata(filename)
print('X的前五行: \n', X[:5,:])
print('y的 前五行: \n', y[:5,])

X的前五行: 
 [[1.       0.97681  0.10723  0.64385  0.29556 ]
 [1.       0.67194  0.2418   0.83075  0.42741 ]
 [1.       0.20619  0.23321  0.81004  0.98691 ]
 [1.       0.51583  0.055814 0.92274  0.75797 ]
 [1.       0.70893  0.10836  0.33951  0.77058 ]]
y的 前五行: 
 [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [112]:
np.array([1]) == np.array([1.0])

array([ True])

In [118]:
# 原始的pla算法
w = np.zeros((X.shape[1], 1))
def pla(X,y,w, iterations=100):
    m = X.shape[0]
    feature_num = X.shape[1]
    num = 0
    for iter in range(iterations):
        flag = 1
        for i in range(m):
            sig = X[i:i+1,:] @ w
            sig = np.sign(sig[0])
           # print(sig)
            if not sig:
                sig =-1
            if sig != int(y[i,0]):
                flag =0
                w = w + X[i:i+1,:].T *y[i,0]
                num+=1
        if flag:
            break
    return num, w
pla(X,y,w)         

(45, array([[-3.       ],
        [ 3.0841436],
        [-1.583081 ],
        [ 2.391305 ],
        [ 4.5287635]]))

## 感知机2 从寻找第一个错误开始

In [104]:
w = np.zeros((X.shape[1], 1))
def pla2(X,y,w, iterations  =100):
    num = 0
    prvpos = 0
    while(iterations):
        yhat = np.sign(X @ w)
        yhat[np.where(yhat ==0)] = -1
        index = np.where(yhat != y)[0]
        #print(index)
        if not index.any():
            break
        if not index[index >= prvpos].any():
            prvpos = 0
        pos = index[ index >= prvpos][0]
        prvpos = pos
        
        w =w + y[pos:pos+1,0]* X[pos:pos+1,:].T
        
        num+=1
        iterations -=1
    return num, w
pla2(X,y,w)  

(39, array([[-3.      ],
        [ 2.35262 ],
        [-1.614215],
        [ 2.830732],
        [ 3.853171]]))

## Q16  运行两千次

In [122]:
total = 0
total2 = 0
for i in range(2000):
    w = np.zeros((X.shape[1],1))
    randseed = np.random.permutation(X.shape[0])
    Xrand = X[randseed,:]
    yrand = y[randseed, 0:1]
    num,_ = pla2(Xrand,yrand,w)
    total += num
    w = np.zeros((X.shape[1],1))
    num2,_= pla2(Xrand,yrand,w)
    total2 += num2

print('2000次平均迭代次数1：',total/2000)
print('2000次平均迭代次数2：', total2/2000)
    

2000次平均迭代次数1： 40.2875
2000次平均迭代次数2： 40.2875


## Q18

In [124]:
filename2 = 'hw1_18_train.dat'
X_train,y_train = loaddata(filename2)

In [125]:
test_file ='hw1_18_test.dat'
X_test,y_test = loaddata(test_file)

In [126]:
rol_train = X_train.shape[0]
col_train = X_train.shape[1]
w2 = np.zeros((col_train,1))

In [123]:
def mistake(yhat,y):
    rol = y.shape[0]
    return np.sum(yhat != y)/rol

In [141]:
def pocket(X,y,theta,iternums, eta):
    yhat =  np.sign(X @ theta)
    yhat[np.where(yhat == 0)] =-1
    index = np.where(yhat != y)[0]
    errold = mistake(yhat,y)
    thetabest = np.zeros((X.shape[1],1))
    for i in range(iternums):
        if not index.any():
            break
        pos = index[np.random.permutation(len(index))][0]
        theta = theta + eta* X[pos:pos+1,:].T *y[pos, 0]
        yhat = np.sign(X @ theta)
        yhat[np.where(yhat==0)] = -1
        index = np.where(yhat!=y)[0]
        errnew = mistake(yhat,y)
        if errnew < errold:
            thetabest = theta.copy()
            errold = errnew
    return thetabest, theta
        

In [142]:
# Q18
total = 0
for i in range(2000):
    theta = np.zeros((X_train.shape[1],1))
    randpos = np.random.permutation(rol_train)
    Xrand_train = X_train[randpos]
    yrand_train  = y_train[randpos]
    theta_best,_ = pocket(X_train, y_train, theta, 50, 1)
    yhat_train = np.sign(X_test @ theta_best)
    yhat_train[np.where(yhat_train == 0)] = -1
    err = mistake(yhat_train, y_test)
    total +=err
print('迭代两千次后的平均错误率：', total/2000)

迭代两千次后的平均错误率： 0.1328629999999994


In [143]:
# Q19
total = 0
for i in range(2000):
    theta = np.zeros((X_train.shape[1],1))
    randpos = np.random.permutation(rol_train)
    Xrand_train = X_train[randpos]
    yrand_train  = y_train[randpos]
    _,theta_best = pla2(X_train, y_train, theta, 50)
    yhat_train = np.sign(X_test @ theta_best)
    yhat_train[np.where(yhat_train == 0)] = -1
    err = mistake(yhat_train, y_test)
    total +=err
print('迭代两千次后的平均错误率：', total/2000)

迭代两千次后的平均错误率： 0.2019999999999984


In [144]:
#Q20

total = 0
for i in range(2000):
    theta = np.zeros((X_train.shape[1],1))
    randpos = np.random.permutation(rol_train)
    Xrand_train = X_train[randpos]
    yrand_train  = y_train[randpos]
    theta_best,_ = pocket(X_train, y_train, theta, 100, 1)
    yhat_train = np.sign(X_test @ theta_best)
    yhat_train[np.where(yhat_train == 0)] = -1
    err = mistake(yhat_train, y_test)
    total +=err
print('迭代两千次后的平均错误率：', total/2000)

迭代两千次后的平均错误率： 0.11610600000000024
